In [1]:
import numpy as np
import random as rd

In [6]:
class env:
    def __init__(self):
        self.nS = 2
        self.nA = 2
        self.reward = np.matrix([[1,0],[0,1]])
        self.P = {0: np.matrix([[0.5,0.5],[0.5,0.5]]), 1: np.matrix([[0.5,0.5],[0.5,0.5]])}
    
    def offline_set(self, N, k):
        offline_set = {}
        for i in range(N):
            result = []
            for j in range(k):
                if j == 0:
                    state = rd.randint(0,self.nS-1)
                else:
                    num = rd.random()
                    target = self.P[result[len(result)-3]][result[len(result)-2],:]
                    if num <= target[0,0]:
                        state = 0
                    else:
                        state = 1
                action = rd.randint(0,self.nA-1)
                reward = self.reward[int(state), int(action)]
                result.append(int(state))
                result.append(int(action))
                result.append(reward)
            offline_set[i] = result
        return offline_set

In [59]:
env1 = env()
data = env1.offline_set(10000,3)

In [14]:
def extract_history(record, n):
    result = ''
    i = (n-1)*3 + 1
    for j in range(i):
        result += str(record[j])
    return result

In [72]:
def DTR_OWL(data, N):
    DTR = {'0':0, '1':0, '0010': 0, '0011': 0, '0100': 0, '0101': 0, '1000': 0, '1001': 0, '1110': 0, '1111': 0, '0010010':0, 
      '0010011':0, '0010100':0, '0010101':0, '0011000':0, '0011001':0, '0011110':0, '0011111':0, '0100010':0,'0100011':0,
      '0100100': 0, '0100101': 0, '0101000':0, '0101001': 0, '0101110':0, '0101111':0, '1000010':0, '1000011':0, '1000100':0,
      '1000101':0, '1001000': 0, '1001001':0, '1001110':0, '1001111':0, '1110010':0,'1110011':0, '1110100':0, '1110101':0,
      '1111000':0, '1111001':0, '1111110':0, '111111':0}
    ##t = 3
    his_3 = ['0010010', '0010011', '0010100', '0010101', '0011000', '0011001', '0011110', '0011111', '0100010', '0100011',
      '0100100', '0100101',  '0101000', '0101001', '0101110', '0101111', '1000010', '1000011', '1000100',
      '1000101', '1001000', '1001001', '1001110', '1001111', '1110010', '1110011', '1110100', '1110101',
      '1111000', '1111001', '1111110', '1111111']
    his_3_re = []
    his_3_0_weight = []
    his_3_1_weight = []
    for i in his_3:
        count_0 = 0
        count_0_re = 0
        count_1 = 0
        count_1_re = 0
        for j in range(N):
            if extract_history(data[j],3) == i:
                if data[j][len(data[j])-2] == 0:
                    count_0 += 1
                    count_0_re += data[j][len(data[j])-1]
                else:
                    count_1 += 1
                    count_1_re += data[j][len(data[j])-1]
        count_0_re = (count_0_re/(count_0/(count_0+count_1)))/(count_0+count_1)
        count_1_re = (count_1_re/(count_1/(count_0+count_1)))/(count_0+count_1)
        if count_0_re >= count_1_re:
            his_3_re.append(0)
        else:
            his_3_re.append(1)
        his_3_0_weight.append(count_0/(count_0+count_1))
        his_3_1_weight.append(count_1/(count_0+count_1))
    
    ##t=2
    his_2 = ['0010', '0011', '0100', '0101', '1000', '1001', '1110', '1111']
    his_2_re = []
    his_2_0_weight = []
    his_2_1_weight = []
    for i in his_2:
        count_0 = 0
        count_0_re = 0
        count_1 = 0
        count_1_re = 0
        for j in range(N):
            if extract_history(data[j],2) == i:
                if data[j][len(data[j])-5] == 0:
                    count_0 += 1
                    for k in range(len(his_3)):
                        if extract_history(data[j],3) == his_3[k]:
                            policy = his_3_re[k]
                            if policy == 0:
                                policy_weight = his_3_0_weight[k]
                            else:
                                policy_weight = his_3_1_weight[k]
                    if data[j][len(data[j])-2] == policy:
                        count_0_re += (data[j][len(data[j])-4]+data[j][len(data[j])-1])/policy_weight
                    
                else:
                    count_1 += 1
                    for k in range(len(his_3)):
                        if extract_history(data[j],3) == his_3[k]:
                            policy = his_3_re[k]
                            if policy == 0:
                                policy_weight = his_3_0_weight[k]
                            else:
                                policy_weight = his_3_1_weight[k]
                    if data[j][len(data[j])-2] == policy:
                        count_1_re += (data[j][len(data[j])-4]+data[j][len(data[j])-1])/policy_weight
        count_0_re = (count_0_re/(count_0/(count_0+count_1)))/(count_0+count_1)
        count_1_re = (count_1_re/(count_1/(count_0+count_1)))/(count_0+count_1)
        if count_0_re >= count_1_re:
            his_2_re.append(0)
        else:
            his_2_re.append(1)
        his_2_0_weight.append(count_0/(count_0+count_1))
        his_2_1_weight.append(count_1/(count_0+count_1))
    ##t=1
    his_1 = ['0', '1']
    his_1_re = []
    his_1_0_weight = []
    his_1_1_weight = []
    for i in his_1:
        count_0 = 0
        count_0_re = 0
        count_1 = 0
        count_1_re = 0
        for j in range(N):
            if extract_history(data[j],1) == i:
                if data[j][len(data[j])-8] == 0:
                    count_0 += 1
                    for k in range(len(his_3)):
                        if extract_history(data[j],3) == his_3[k]:
                            policy = his_3_re[k]
                            if policy == 0:
                                policy_weight = his_3_0_weight[k]
                            else:
                                policy_weight = his_3_1_weight[k]
                    for k in range(len(his_2)):
                        if extract_history(data[j],2) == his_2[k]:
                            policy2 = his_2_re[k]
                            if policy2 == 0:
                                policy2_weight = his_2_0_weight[k]
                            else:
                                policy2_weight = his_2_1_weight[k]
                    
                    if data[j][len(data[j])-2] == policy and data[j][len(data[j])-5] == policy2:
                        count_0_re += (data[j][len(data[j])-7]+data[j][len(data[j])-4]+data[j][len(data[j])-1])/(policy_weight*policy2_weight)
                    
                else:
                    count_1 += 1
                    for k in range(len(his_3)):
                        if extract_history(data[j],3) == his_3[k]:
                            policy = his_3_re[k]
                            if policy == 0:
                                policy_weight = his_3_0_weight[k]
                            else:
                                policy_weight = his_3_1_weight[k]
                    for k in range(len(his_2)):
                        if extract_history(data[j],2) == his_2[k]:
                            policy2 = his_2_re[k]
                            if policy2 == 0:
                                policy2_weight = his_2_0_weight[k]
                            else:
                                policy2_weight = his_2_1_weight[k]
                    if data[j][len(data[j])-2] == policy and data[j][len(data[j])-5] == policy2:
                         count_1_re += (data[j][len(data[j])-7]+data[j][len(data[j])-4]+data[j][len(data[j])-1])/(policy_weight*policy2_weight)
                            
        count_0_re = (count_0_re/(count_0/(count_0+count_1)))/(count_0+count_1)
        count_1_re = (count_1_re/(count_1/(count_0+count_1)))/(count_0+count_1)
        if count_0_re >= count_1_re:
            his_1_re.append(0)
        else:
            his_1_re.append(1)
        his_1_0_weight.append(count_0/(count_0+count_1))
        his_1_1_weight.append(count_1/(count_0+count_1))
    for item in range(len(his_1)):
        DTR[his_1[item]] = his_1_re[item]
    for item in range(len(his_2)):
        DTR[his_2[item]] = his_2_re[item]
    for item in range(len(his_3)):
        DTR[his_3[item]] = his_3_re[item]
    return DTR
        
        
        
                
            
        

In [73]:
DTR = DTR_OWL(data, 1000)

In [74]:
DTR

{'0': 0,
 '1': 1,
 '0010': 0,
 '0011': 1,
 '0100': 0,
 '0101': 1,
 '1000': 0,
 '1001': 1,
 '1110': 0,
 '1111': 1,
 '0010010': 0,
 '0010011': 1,
 '0010100': 0,
 '0010101': 1,
 '0011000': 0,
 '0011001': 1,
 '0011110': 0,
 '0011111': 1,
 '0100010': 0,
 '0100011': 1,
 '0100100': 0,
 '0100101': 1,
 '0101000': 0,
 '0101001': 1,
 '0101110': 0,
 '0101111': 1,
 '1000010': 0,
 '1000011': 1,
 '1000100': 0,
 '1000101': 1,
 '1001000': 0,
 '1001001': 1,
 '1001110': 0,
 '1001111': 1,
 '1110010': 0,
 '1110011': 1,
 '1110100': 0,
 '1110101': 1,
 '1111000': 0,
 '1111001': 1,
 '1111110': 0,
 '111111': 0,
 '1111111': 1}